# Setup and basic objects

Get started with EnOSlib on Grid'5000.

---

- Website: https://discovery.gitlabpages.inria.fr/enoslib/index.html
- Instant chat: https://framateam.org/enoslib
- Source code: https://gitlab.inria.fr/discovery/enoslib

---

This is the first notebooks of a series that will let you discover the main features of EnOSlib/Grid'5000.

If you want to actually execute them you'll need to setup your environment properly.
We sum up here the different steps to achieve this process.

1. Get a Grid'5000 account
    - Register using this [page](https://www.grid5000.fr/w/Grid5000:Get_an_account).
      Pay attention to the fact that uploading a SSH key (public part) is mandatory to perform any EnOSlib action.
    - Make sure the SSH connection is ready. You can follow this [tutorial](https://www.grid5000.fr/w/Getting_Started).
2. Make sure EnOSlib is available in your notebook environment
    - Follow the steps [here](https://discovery.gitlabpages.inria.fr/enoslib/tutorials/grid5000.html#installation).
      Using a virtualenv is the way to go, make sure to use one.



## Testing the import

In [1]:
import enoslib as en

## Resources abstractions

In this notebook we won't execute anything remotely, instead we'll just cover some basic abstractions provided by the library.
We start with the abstractions of the resources (machines and networks that are usually given by an infrastructure)


### Host

An host is anything we can connect to and act on. Most of the time it corresponds to a machine reachable through SSH.
The datastructure reflects this.

Usually you don't instantiate hosts manually, instead they are brought to you by EnOSlib (because most likely they depend on a scheduler decision like OAR on Grid'5000).


In [2]:
bare_host = en.Host("192.168.0.1")
host_with_alias = en.Host("192.168.0.2", alias="one_alias")
host_with_alias_and_username = en.Host("192.168.0.3", alias="one_alias", user="foo")

In [3]:
bare_host

Host(address='192.168.0.1', alias='192.168.0.1', user=None, keyfile=None, port=None, extra={}, net_devices=set())

In [4]:
host_with_alias

Host(address='192.168.0.2', alias='one_alias', user=None, keyfile=None, port=None, extra={}, net_devices=set())

In [5]:
host_with_alias_and_username

Host(address='192.168.0.3', alias='one_alias', user='foo', keyfile=None, port=None, extra={}, net_devices=set())

The local machine can be represented by an instance of the `LocalHost` object. This is a specialization of an `Host`, the connection to this host will be made using sub-processes (instead of SSH). We can see it in the `extra` attribute of the `LocalHost` object. This `extra` attribute is actually interpreted when a "remote" action is triggered on our hosts.

In [6]:
localhost = en.LocalHost()
localhost

LocalHost(address='localhost', alias='localhost', user=None, keyfile=None, port=None, extra={'ansible_connection': 'local'}, net_devices=set())

Other types of Hosts are possible. The library has a `DockerHost` which represents a docker container we want to reach using the docker TCP protocol. One needs to specify where this container is running by passing an host instance.

In [7]:
docker_host = en.DockerHost("alias", "container_name", host_with_alias_and_username)
docker_host

DockerHost(address='container_name', alias='alias', user='foo', keyfile=None, port=None, extra={'ansible_connection': 'docker', 'ansible_docker_extra_args': '-H ssh://foo@192.168.0.3', 'mitogen_via': 'foo@192.168.0.3'}, net_devices=set())

The above `extra` field suggest that the connection to this docker container will be made through an ssh jump to the remote host hosting the container.
This will be done transparently by the library anyway.

---

### Roles

A common pratice when experimenting, especially with distributed applications, is to form logical group of machines.
Indeed, during an experiment your hosts will serve different purposes: some will host the system you are studying while other will install third party tools to inject some load, observe ...

A natural way of configuring differently several sets of hosts is to tag them and group them according to their tags.

The `Roles` datastructure serves this purpose: it lets you group your hosts based on tags. It follow a `dict-like` interface.

In [8]:
h1 = en.Host("10.0.0.1")
h2 = en.Host("10.0.0.2")
h3 = en.Host("10.0.0.3")
roles = en.Roles()
roles["tag1"] = [h1, h2]
roles["tag2"] = [h3]
roles["tag3"] = [h2, h3]

roles

{'tag1': [Host(address='10.0.0.1', alias='10.0.0.1', user=None, keyfile=None, port=None, extra={}, net_devices=set()), Host(address='10.0.0.2', alias='10.0.0.2', user=None, keyfile=None, port=None, extra={}, net_devices=set())], 'tag2': [Host(address='10.0.0.3', alias='10.0.0.3', user=None, keyfile=None, port=None, extra={}, net_devices=set())], 'tag3': [Host(address='10.0.0.2', alias='10.0.0.2', user=None, keyfile=None, port=None, extra={}, net_devices=set()), Host(address='10.0.0.3', alias='10.0.0.3', user=None, keyfile=None, port=None, extra={}, net_devices=set())]}

### Network and Networks

`Network` and `Networks`  are the same as `Host` and `Roles` but for networks:

- `Network` represent a single Network
- `Networks` represent a "Roles" of Network: networks indexed by their tags
.

Networks are usually given by an infrastructure and thus you won't really instantiate `Network` nor `Networks` by yourself.
More precisely there exists a specific subclass of `Network` per infrastructure which will be returned automatically by EnOSlib when needed.

Moreover `Network` datastructure isn't exposed in EnOSlib at the top level, let's see however how a `DefaultNetwork` can look like. A `DefaultNetwork` is a very common abstraction of a network that allows to represent a basic network with optionnally a pool of free ips/macs address. For instance a subnet or a vlan on Grid5000 are represented by a specific `DefaultNetwork`.

In [9]:
from enoslib.objects import DefaultNetwork

In [10]:
one_network = DefaultNetwork("192.168.1.0/24")
one_network_with_a_pool_of_ips = DefaultNetwork("192.168.1.0/24", ip_start="192.168.1.10", ip_end="192.168.1.100") 

In [11]:
one_network

In [12]:
one_network_with_a_pool_of_ips

In [13]:
# get one free ip
ip_gen = one_network_with_a_pool_of_ips.free_ips
next(ip_gen)

IPv4Address('192.168.1.10')

In [14]:
# get another one
next(ip_gen)

IPv4Address('192.168.1.11')

## Providers (and their configurations)

EnOSlib uses `Provider`s to ... provide resources. 
`Provider`s  let the user get ownership of some resources (for the time of the experiment) in good shape (e.g access granted, network configured ...). 
They transform an abstract `Configuration` to `Roles, Networks` : 


$$Configuration \xrightarrow{provider} Roles, Networks$$


There are different providers in EnOSlib: 


- **Vbox/KVM** to work with locally hosted virtual machines
- **Openstack/Chameleon** to work with bare-metal resources hosted in the Chameleon platform
- **FiT/IOT lab** to work with sensors or low profile machines
- **Grid'5000** to get bare-metal resources from G5k.<br/>
  There are also some composite providers that sit on top of the Grid'5000 provider
  
    - **VmonG5k** to work with virtual machines on Grid'5000**
    - **Distem** to work with lxc containers on Grid'5000**



### Configurations

A `Provider` must be fed with a `Configuration`.  `Configuration` objects are specific to each provider.

You can build them from a dictionnary (e.g from a yaml/json file) or programmatically. For instance the schema for Grid'5000 is [here](https://discovery.gitlabpages.inria.fr/enoslib/apidoc/infra.html#g5k-schema).

In this section, we'll only build some configurations (No resource will be reserved on Grid'5000)

In [13]:
import enoslib as en

# An empty configuration isn't really useful but let you see
# some of the default parameters
# Note that by default the job_type is set to deploy == the env_name will be deployed
conf = en.G5kConf()
conf

Conf@0x7f98a43f9c10
{
    "dhcp": true,
    "force_deploy": false,
    "env_name": "debian10-x64-nfs",
    "job_name": "EnOSlib",
    "job_type": [
        "deploy"
    ],
    "key": "/home/msimonin/.ssh/id_rsa.pub",
    "queue": "default",
    "walltime": "02:00:00",
    "resources": {
        "machines": [],
        "networks": []
    }
}

In [14]:
# changing the top level options is done by calling the classmethod `from_settings`
en.G5kConf.from_settings(walltime="10:00:00", job_name="my awesome job")

Conf@0x7f98a4e33190
{
    "dhcp": true,
    "force_deploy": false,
    "env_name": "debian10-x64-nfs",
    "job_name": "my awesome job",
    "job_type": [
        "deploy"
    ],
    "key": "/home/msimonin/.ssh/id_rsa.pub",
    "queue": "default",
    "walltime": "10:00:00",
    "resources": {
        "machines": [],
        "networks": []
    }
}

In [15]:
# the canonical way of getting some machines

In [21]:
prod_network = en.G5kNetworkConf(id="n1", roles=["mynetwork"], site="rennes", type="prod")
conf = (
    en.G5kConf()
        .add_machine(cluster="paravance", nodes=3, roles=["role1", "role2"], primary_network=prod_network)
        .add_machine(cluster="parasilo", nodes=3, roles=["role2", "role3"], primary_network=prod_network)
        .add_network_conf(prod_network)
        # optional, but do some sanity checks on the configuration 
        .finalize()
    )
conf

Conf@0x7f98a43aac10
{
    "dhcp": true,
    "force_deploy": false,
    "env_name": "debian10-x64-nfs",
    "job_name": "EnOSlib",
    "job_type": [
        "deploy"
    ],
    "key": "/home/msimonin/.ssh/id_rsa.pub",
    "queue": "default",
    "walltime": "02:00:00",
    "resources": {
        "machines": [
            {
                "roles": [
                    "role1",
                    "role2"
                ],
                "primary_network": "n1",
                "secondary_networks": [],
                "cluster": "paravance",
                "nodes": 3
            },
            {
                "roles": [
                    "role2",
                    "role3"
                ],
                "primary_network": "n1",
                "secondary_networks": [],
                "cluster": "parasilo",
                "nodes": 3
            }
        ],
        "networks": [
            {
                "id": "n1",
                "type": "prod",
                "roles": [
                    "mynetwork"
                ],
                "site": "rennes"
            }
        ]
    }
}

In [18]:
# changing to a non-deploy job
# == no deployment will occur, the production environment will be used
prod_network = en.G5kNetworkConf(id="n1", roles=["mynetwork"], site="rennes", type="prod")
conf = (
    en.G5kConf.from_settings(job_type=["allow_classic_ssh"])
        .add_machine(cluster="paravance", nodes=3, roles=["role1", "role2"], primary_network=prod_network)
        .add_machine(cluster="parasilo", nodes=3, roles=["role2", "role3"], primary_network=prod_network)
        .add_network_conf(prod_network)
        # optional, but do some sanity checks on the configuration 
        .finalize()
    )
conf

Conf@0x7f98a4390a90
{
    "dhcp": true,
    "force_deploy": false,
    "env_name": "debian10-x64-nfs",
    "job_name": "EnOSlib",
    "job_type": [
        "allow_classic_ssh"
    ],
    "key": "/home/msimonin/.ssh/id_rsa.pub",
    "queue": "default",
    "walltime": "02:00:00",
    "resources": {
        "machines": [
            {
                "roles": [
                    "role1",
                    "role2"
                ],
                "primary_network": "n1",
                "secondary_networks": [],
                "cluster": "paravance",
                "nodes": 3
            }
        ],
        "networks": [
            {
                "id": "n1",
                "type": "prod",
                "roles": [
                    "mynetwork"
                ],
                "site": "rennes"
            }
        ]
    }
}

In [26]:
# Using a secondary networks
prod_network = en.G5kNetworkConf(id="n1", roles=["mynetwork"], site="rennes", type="prod")
kavlan_network = en.G5kNetworkConf(id="n2", roles=["myprivate"], site="rennes", type="kavlan")
conf = (
    en.G5kConf()
        .add_machine(cluster="paravance", nodes=3, roles=["role1", "role2"], primary_network=prod_network, secondary_networks=[kavlan_network])
        .add_machine(cluster="parasilo", nodes=3, roles=["role2", "role3"], primary_network=prod_network, secondary_networks=[kavlan_network])
        .add_network_conf(prod_network)
        .add_network_conf(kavlan_network)
        # optional, but do some sanity checks on the configuration 
        .finalize()
    )
conf

Conf@0x7f98a43af4d0
{
    "dhcp": true,
    "force_deploy": false,
    "env_name": "debian10-x64-nfs",
    "job_name": "EnOSlib",
    "job_type": [
        "deploy"
    ],
    "key": "/home/msimonin/.ssh/id_rsa.pub",
    "queue": "default",
    "walltime": "02:00:00",
    "resources": {
        "machines": [
            {
                "roles": [
                    "role1",
                    "role2"
                ],
                "primary_network": "n1",
                "secondary_networks": [
                    "n2"
                ],
                "cluster": "paravance",
                "nodes": 3
            },
            {
                "roles": [
                    "role2",
                    "role3"
                ],
                "primary_network": "n1",
                "secondary_networks": [
                    "n2"
                ],
                "cluster": "parasilo",
                "nodes": 3
            }
        ],
        "networks": [
            {
                "id": "n1",
                "type": "prod",
                "roles": [
                    "mynetwork"
                ],
                "site": "rennes"
            },
            {
                "id": "n2",
                "type": "kavlan",
                "roles": [
                    "myprivate"
                ],
                "site": "rennes"
            }
        ]
    }
}

### Discussion and references

- Many configurations options are possible. [The documentation](https://discovery.gitlabpages.inria.fr/enoslib/tutorials/grid5000.html) will show you some more.
- In EnOSlib `Roles` and `Networks` don't really depend on the provider that produced them. In other words you can substitute one provider's configuration to another one easily without changing the artifact code.